# Модуль 4.0 — Предобработка данных для RAG

**Цель:** понять, как готовить данные до загрузки в RAG, чтобы поиск и ответы были качественными.

**Что сделаем:**
- разберём, зачем нужна предобработка
- перечислим лучшие практики
- покажем минимальный пример очистки и нормализации

## Зачем нужна предобработка

RAG зависит от качества текста. Если в данных шум, дубли и ошибки, то ретривер найдёт «мусорные» куски, а LLM будет отвечать неточно.

**Что даёт предобработка:**
- повышает качество поиска
- снижает количество ложных совпадений
- улучшает связность ответа
- уменьшает стоимость (меньше лишних токенов)

**Когда это критично:**
- PDF/сканы с плохим OCR
- веб‑страницы с навигацией и «лишним» HTML
- базы знаний с дубликатами и устаревшими статьями

## Лучшие практики предобработки

**1) Очистка шума**
- удалить навигацию, футеры, повторяющиеся элементы
- убрать лишние пробелы и «служебные» символы

**2) Нормализация**
- привести кавычки/тире к единому виду
- нормализовать регистр, если это не ломает смысл

**3) Дедупликация**
- убрать дублированные абзацы и копии документов
- хранить уникальные версии с датой обновления

**4) Разметка метаданных**
- источник, дата, раздел, язык
- это улучшает фильтрацию и поиск

**5) Защита данных**
- маскирование PII (телефоны, email, имена)
- не индексировать секреты и ключи

**6) Языковая проверка**
- разделять документы по языкам
- избегать смешивания, если модель не мультиязычная

## Минимальный пример очистки и нормализации

Ниже — базовый пример для `.txt`. Для PDF/HTML потребуется отдельный парсинг и извлечение текста.

In [ ]:
import re
from pathlib import Path

RAW_DIR = Path("./raw")
CLEAN_DIR = Path("./clean")
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

# Пример: функция очистки

def clean_text(text: str) -> str:
    # Удаляем лишние пробелы и переносы
    text = re.sub(r"\s+", " ", text).strip()
    # Нормализуем кавычки и тире
    text = text.replace("«", '"').replace("»", '"')
    text = text.replace("–", "-").replace("—", "-")
    return text

# Обрабатываем все txt файлы из ./raw
for path in RAW_DIR.glob("*.txt"):
    raw = path.read_text(encoding="utf-8", errors="ignore")
    cleaned = clean_text(raw)
    (CLEAN_DIR / path.name).write_text(cleaned, encoding="utf-8")

print("Готово. Чистые файлы в:", CLEAN_DIR)

## Практические рекомендации

- **Храните сырьё отдельно** (`./raw`) и чистые данные отдельно (`./clean`).
- **Версионируйте** датой: это помогает откатываться.
- **Проверяйте качество OCR** — это главный источник ошибок в RAG.
- **Удаляйте дубли** на уровне абзацев, не только файлов.
- **Сохраняйте метаданные** (источник/дата/раздел), чтобы фильтровать результаты.
- **Не индексируйте секреты** — лучше маскировать до загрузки.